In [1]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import time

In [2]:
cap =  cv2.VideoCapture(0)
ref_img = False 

while cap.isOpened():
    
    success, frame_img = cap.read()
    # frame = cv2.rotate(frame, cv2.ROTATE_180)

    if not success:
        print("Ignoring empty camera frame.")
        break

    if not ref_img:    
        cv2.imshow('Pressed Key Frame', frame_img)
        
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break

    cv2.waitKey(1)
    if cv2.getWindowProperty('Pressed Key Frame', cv2.WND_PROP_VISIBLE) < 1:
        break
    
cap.release()
cv2.destroyAllWindows()

In [4]:
# Initialize blank images for color exploration and selection
color_explore = np.zeros((150, 150, 3), dtype=np.uint8)
color_selected = np.zeros((150, 150, 3), dtype=np.uint8)

def show_color(event, x, y, flags, param):
    global frame_img  # Ensure you have access to the image variable
    
    # Ensure x, y are within bounds of the image
    if y >= frame_img.shape[0] or x >= frame_img.shape[1]:
        return

    # Convert the BGR image to HSV
    hsv_img = cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV)

    # Extract HSV color components from the image at cursor position
    H, S, V = hsv_img[y, x]

    # Convert single HSV value to full image for display
    hsv_color = np.uint8([[[H, S, V]]])
    bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)

    # Update color_explore with the color under cursor in BGR for display
    color_explore[:] = bgr_color

    # If left mouse button pressed, update color_selected and print the HSV color values
    if event == cv2.EVENT_LBUTTONDOWN:
        color_selected[:] = bgr_color  # Display in BGR
        print("Selected HSV Color: ({}, {}, {})".format(H, S, V))

# Create windows for color exploration and selected color display
cv2.namedWindow('color_explore')
cv2.resizeWindow('color_explore', 150, 150)

cv2.namedWindow('color_selected')
cv2.resizeWindow('color_selected', 150, 150)

# Create window for displaying the sample image
cv2.namedWindow('image')

# Load or access your image here
# For example:
# img = cv2.imread('path_to_your_image.jpg')
# Here, img should be the frame from your video or any image you're working with

# Assign the mouse callback function to the 'image' window
cv2.setMouseCallback('image', show_color)

# Main loop for live update of the color exploration and selection
while True:
    cv2.imshow('image', frame_img)
    cv2.imshow('color_explore', color_explore)
    cv2.imshow('color_selected', color_selected)

    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Clean up windows
cv2.destroyAllWindows()


Selected HSV Color: (33, 76, 207)
Selected HSV Color: (161, 116, 228)


In [5]:
def apply_threshold_hsv(roi, hsv_color, threshold):
    """Apply color thresholding in HSV color space to isolate specific color ranges in the ROI."""
    lower_bound = np.array([max(0, hsv_color[0] - threshold), max(0, hsv_color[1] - threshold), max(0, hsv_color[2] - threshold)])
    upper_bound = np.array([min(180, hsv_color[0] + threshold), min(255, hsv_color[1] + threshold), min(255, hsv_color[2] + threshold)])
    mask = cv2.inRange(roi, lower_bound, upper_bound)
    return mask

In [7]:
def find_clusters(mask):
    """Find clusters in the mask using DBSCAN."""
    
    y_coord, x_coord = np.where(mask != 0)
    if len(y_coord) == 0:
        return {}  # Return an empty dict if no points found
    
    coord_array = np.stack((y_coord, x_coord), axis=-1)
    sorted_array = coord_array[coord_array[:, 1].argsort()]
    dbscan = DBSCAN(eps=5, min_samples=10)
    clusters = dbscan.fit_predict(sorted_array)

    cluster_dict = {}
    for point, cluster_idx in zip(sorted_array, clusters):
        if cluster_idx != -1:
            cluster_dict.setdefault(cluster_idx, []).append(point.tolist())
            
    return cluster_dict

def filter_noise_clusters(cluster_dict, size_threshold):
    """
    Filters clusters based on a minimum size threshold.

    Parameters:
    - cluster_dict (dict): A dictionary where each key represents a cluster index,
      and the value is a list of points belonging to that cluster.
    - size_threshold (int): The minimum number of points a cluster must have to be included.

    Returns:
    - dict: A new dictionary containing only the clusters that meet the size threshold.
    """
    
    filtered_clusters = {}
    for key, points in cluster_dict.items():
        if len(points) > size_threshold:
            filtered_clusters[key] = points
            
    return filtered_clusters

In [8]:
def highlight_clusters(frame_roi, cluster_dict, error_keys, color=(0, 0, 255)):
    """
    Highlights specified clusters in a region of interest by changing their pixel colors.

    Parameters:
    - frame_roi (numpy.ndarray): The region of interest from the frame where clusters are to be highlighted.
    - cluster_dict (dict): A dictionary containing clusters with their points. Each key in the dictionary
      represents a cluster index, and the value is a list of points (row, column pairs) belonging to that cluster.
    - error_keys (list): A list of keys indicating which clusters in the cluster_dict should be highlighted.
    - color (tuple): The BGR color value to use for highlighting. Default is red (0, 0, 255).

    Returns:
    - numpy.ndarray: The modified region of interest with specified clusters highlighted.
    """
    for key in error_keys:
        if key in cluster_dict:  # Check if the key exists in the cluster dictionary
            for row, col in cluster_dict[key]:
                frame_roi[row, col] = color
    return frame_roi

In [ ]:
def reference_frame(frame, mask_bound, hsv_color_1, hsv_color_2, threshold = 40):
    
    x1, y1, x2, y2 = mask_bound
    roi = frame[y1:y2, x1:x2]
    roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Apply threshold in HSV color space
    mask_1 = apply_threshold_hsv(roi_hsv, hsv_color_1, threshold)
    roi[mask_1 != 0] = [0, 255, 255]
    cluster_dict_1 = find_clusters(mask_1)
    cluster_dict_1 = filter_noise_clusters(cluster_dict_1, 100)

    mask_2 = apply_threshold_hsv(roi_hsv, hsv_color_2, threshold)
    roi[mask_2 != 0] = [60, 255, 255]
    cluster_dict_2 = find_clusters(mask_2)
    cluster_dict_2 = filter_noise_clusters(cluster_dict_2, 100)

    # Convert clusters to np.array for potential performance improvements
    for cluster_idx in cluster_dict_1:
        cluster_dict_1[cluster_idx] = np.array(cluster_dict_1[cluster_idx])

    for cluster_idx in cluster_dict_2:
        cluster_dict_2[cluster_idx] = np.array(cluster_dict_2[cluster_idx])

    return roi_hsv, cluster_dict_1, cluster_dict_2

In [39]:
def filter_keys(cluster_dict, roi, inf_roi, error_bounds):
    """
    Calculates error percentages for clusters and filters keys based on error bounds.

    Parameters:
    - cluster_dict (dict): Clusters to analyze, where each key is a cluster index, and the value is a list of points.
    - roi (numpy.ndarray): The reference region of interest from the original frame.
    - inf_roi (numpy.ndarray): The inference region of interest from the compared frame.
    - error_bounds (tuple): A tuple containing two lists, the first for lower bounds and the 
      second for upper bounds of error percentages for filtering. Each list's length should match the number of clusters.

    Returns:
    - tuple: (error_keys, error_percentages)
        - error_keys (list): The keys of clusters that fall within the specified error bounds.
        - error_percentages (list): The error percentages of all clusters.
    """
    error_keys = []
    error_percentages = []
    error_lower_bound, error_upper_bound = error_bounds

    for index, (key, cluster) in enumerate(cluster_dict.items()):
        error_count = sum(1 for row_ref, col_ref in cluster if not np.array_equal(roi[row_ref, col_ref], inf_roi[row_ref, col_ref]))
        total_comparisons = len(cluster)

        error_percentage = (error_count / total_comparisons) * 100 if total_comparisons > 0 else 0
        error_percentages.append(error_percentage)
        
        if total_comparisons > 0 and error_lower_bound[index] < error_percentage < error_upper_bound[index]:
            error_keys.append(key)

    return error_keys

In [42]:
def inference_frame(inf_frame, mask_bound, hsv_color_1, hsv_color_2, cluster_dict_1, cluster_dict_2, roi, threshold=40, error_bound_1=(), error_bound_2=()):
    x1, y1, x2, y2 = mask_bound
    inf_roi = inf_frame[y1:y2, x1:x2]
    inf_hsv = cv2.cvtColor(inf_roi, cv2.COLOR_BGR2HSV)

    # black
    mask_1 = apply_threshold_hsv(inf_hsv, hsv_color_1, threshold)
    inf_hsv[mask_1 != 0] = [0, 255, 255]

    # white
    mask_2 = apply_threshold_hsv(inf_hsv, hsv_color_2, threshold)
    inf_hsv[mask_2 != 0] = [60, 255, 255]

    error_bound_1 = ([6,6,6,6,6,6,6,6,6], [20,20,20,20,20,20,20,20,20])
    error_bound_2 = ([6,6,6,6,6,6,6,6,6], [20,20,20,20,20,20,20,20,20])
    
    black_error_keys = filter_keys(cluster_dict_1, roi, inf_hsv, error_bound_1)
    white_error_keys = filter_keys(cluster_dict_2, roi, inf_hsv, error_bound_2)

    return inf_hsv, black_error_keys, white_error_keys

In [35]:
white_keys = ['C3', 'D3', 'E3', 'F3', 'G3', 'A3', 'B3', 'C4']
black_keys = ['C#3', 'D#3', 'F#3', 'G#3', 'A#3']

def encode_to_scale(values, scale):
    encoded_notes = []
    scale_length = len(scale)
    for value in values:
        # Map each value to a note in the scale
        note = scale[value % scale_length]
        encoded_notes.append(note)
    return encoded_notes

In [45]:
cap =  cv2.VideoCapture(0)
ref_img = False

black_tag = (33, 76, 207)
white_tag = (161, 116, 228)
mask_bound = (0, 265, 640, 452)
threshold = 40

while cap.isOpened():
    
    success, frame_img = cap.read()

    if not success:
        print("Ignoring empty camera frame.")
        break

    if not ref_img:    
        cv2.imshow('Pressed Key Frame', frame_img)
        
        if cv2.waitKey(1) & 0xFF == ord('s'):
            print("Reference Frame Captured")
            roi, cluster_dict_1, cluster_dict_2 = reference_frame(frame_img, mask_bound, black_tag, white_tag)
            ref_img = True
            frame_img = roi

    elif(ref_img):
        height, width, _ = frame_img.shape
        mask_bound = (0, 0, width, height)
        
        frame_roi, error_keys_black, error_keys_white = \
            inference_frame(frame_img, mask_bound, black_tag, white_tag, cluster_dict_1, cluster_dict_2, roi, threshold)
        encoded_notes_white = encode_to_scale(error_keys_white, white_keys)
        encoded_notes_black = encode_to_scale(error_keys_black, black_keys)
        all_notes = encoded_notes_white + encoded_notes_black
        if(all_notes):
            print(all_notes)
        
        print("White Keys: ", error_keys_white)
        print("Black Keys: ", error_keys_black)
        print("=-------------------=")
        frame_roi = cv2.cvtColor(frame_roi, cv2.COLOR_HSV2BGR)
        
        for keys in error_keys_black:
            for i in cluster_dict_1[keys]:
                rows, columns = i
                
                frame_roi[rows][columns][0] = 0
                frame_roi[rows][columns][1] = 0
                frame_roi[rows][columns][2] = 255

        for keys in error_keys_white:
            for i in cluster_dict_2[keys]:
                rows, columns = i
                
                frame_roi[rows][columns][0] = 0
                frame_roi[rows][columns][1] = 255 
                frame_roi[rows][columns][2] = 255
        
        cv2.imshow('Pressed Key Frame', frame_roi)

    cv2.waitKey(1)
    if cv2.getWindowProperty('Pressed Key Frame', cv2.WND_PROP_VISIBLE) < 1:
        break
    
cap.release()
cv2.destroyAllWindows()

Reference Frame Captured
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
White Keys:  []
Black Keys:  []
=-------------------=
Whi

KeyboardInterrupt: 